# move data from one db to another

see: https://github.com/APS-3ID-IXN/ipython-s3blue/issues/2

the packages we'll be using

In [1]:
from databroker import Broker
import datetime, os, shutil, time
import pyRestTable

connect to the database where scans were collected

In [2]:
server = "otz"
otz_config = {
    "description": "heavyweight shared database",
    "metadatastore": {
        "module": "databroker.headersource.mongo",
        "class": "MDS",
        "config": {
            "host": server,
            "port": 27017,
            "database": "metadatastore-production-v1",
            "timezone": "US/Central"
        }
    },
    "assets": {
        "module": "databroker.assets.mongo",
        "class": "Registry",
        "config": {
            "host": server,
            "port": 27017,
            "database": "filestore-production-v1"
        }
    }
}
otz_db = Broker.from_config(otz_config)

Show the scans between the dates mentioned in the [issue](https://github.com/APS-3ID-IXN/ipython-s3blue/issues/2)

In [3]:
headers = otz_db(since='2018-10-20', until='2018-11-06')
for h in headers:
    print(h.start)


Run Start
BLUESKY_VERSION : 1.3.3                                   
EPICS_AR_PORT   : 7002                                    
EPICS_CA_MAX_ARR: 16777216                                
EPICS_EXTENSIONS: /APSshare/epics/extensions              
EPICS_HOST_ARCH : linux-x86_64                            
OPHYD_VERSION   : 1.2.0                                   
beamline_id     : 3-ID                                    
detectors       : ['mca', 'scaler']                       
extents         : [[0, 0], [-0.0005, 0.0005]]             
hints           :
  gridding        : rectilinear                             
  dimensions      : [[['neat_stage_y'], 'primary'], [['neat_stage_x'], 'primary']]
login_id        : s3blue@ni.xray.aps.anl.gov              
motors          : ['neat_stage_y', 'neat_stage_x']        
num_intervals   : 9                                       
num_points      : 10                                      
pid             : 28587                                   
p

All those scans have the key `beamline_id="3-ID"` so get all the scan (headers) that match that key.  Print the results in a nicer table.

In [4]:
tbl = pyRestTable.Table()
tbl.labels = "date/time # uid plan_name".split()
headers = otz_db(beamline_id='3-ID')
for h in headers:
    t_float = h.start["time"]
    dt = datetime.datetime.fromtimestamp(t_float)
    row = [dt, h.start["scan_id"], h.start["uid"], h.start["plan_name"]]
    tbl.addRow(row)
print(tbl)

========================== == ==================================== =============
date/time                  #  uid                                  plan_name    
========================== == ==================================== =============
2018-10-29 03:45:24.167806 48 6ea22d45-7b57-4936-b007-a6129a04a28c rel_grid_scan
2018-10-28 18:56:00.852009 47 b7f9a22f-bff3-4fca-8223-3fe3c13cf04a rel_grid_scan
2018-10-28 18:44:26.594428 46 ab52f840-7919-4925-8c62-790a0bf629ef rel_grid_scan
2018-10-28 16:59:11.531815 45 8b318daa-c3ee-46d3-9d42-15c51f2e7598 rel_grid_scan
2018-10-28 16:53:30.997551 44 a4b7fd59-7875-489a-8a5f-1cc44f311760 rel_grid_scan
2018-10-28 16:50:59.680815 43 8ee33a6f-0a74-43fa-8d1d-99e94e868811 rel_grid_scan
2018-10-28 16:48:00.945385 42 ed1ee892-61a3-4b8a-bb3d-4b108465a9e0 rel_grid_scan
2018-10-28 16:37:31.415016 41 7769bca7-bccd-42de-8459-9676d7dcde86 rel_grid_scan
2018-10-28 16:31:16.187840 40 57e0b82d-939b-4547-8eff-9eaad0401712 rel_grid_scan
2018-10-28 16:28:51.544602 3

-----------

## build a local sqlite database for testing

In [5]:
test_dir = "/tmp/bluesky"
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

test_config = {
    "description": "lightweight personal database",
    "metadatastore": {
        "module": "databroker.headersource.sqlite",
        "class": "MDS",
        "config": {
            "directory": test_dir,
            "timezone": "US/Central"
        }
    },
    "assets": {
        "module": "databroker.assets.sqlite",
        "class": "Registry",
        "config": {
            "dbpath": test_dir + "/database.sql"
        }
    }
}
test_db = Broker.from_config(test_config)

## Export (test)
export from otz_db to test_db

In [6]:
if os.path.exists(test_dir):
    shutil.rmtree(test_dir)
    os.makedirs(test_dir)

otz_db.export(headers, test_db)

OperationalError: duplicate column name: data_NFS

In [7]:
h = list(headers)[-1]
h.descriptors

[{'run_start': '0b481920-1617-49f2-a2a3-05dcccb69dc3',
  'data_keys': {'Time': {'source': 'PV:3idd:scaler1.S1',
    'dtype': 'number',
    'shape': [],
    'precision': 0,
    'units': '',
    'lower_ctrl_limit': 0.0,
    'upper_ctrl_limit': 0.0,
    'object_name': 'scaler'},
   'IC0_D': {'source': 'PV:3idd:scaler1.S2',
    'dtype': 'number',
    'shape': [],
    'precision': 0,
    'units': '',
    'lower_ctrl_limit': 0.0,
    'upper_ctrl_limit': 0.0,
    'object_name': 'scaler'},
   'PIN': {'source': 'PV:3idd:scaler1.S3',
    'dtype': 'number',
    'shape': [],
    'precision': 0,
    'units': '',
    'lower_ctrl_limit': 0.0,
    'upper_ctrl_limit': 0.0,
    'object_name': 'scaler'},
   'NaI': {'source': 'PV:3idd:scaler1.S4',
    'dtype': 'number',
    'shape': [],
    'precision': 0,
    'units': '',
    'lower_ctrl_limit': 0.0,
    'upper_ctrl_limit': 0.0,
    'object_name': 'scaler'},
   'Fwd prompt': {'source': 'PV:3idd:scaler1.S5',
    'dtype': 'number',
    'shape': [],
    'pr

Seems be trouble creating ophyd object names with the data column names that have embedded whitespace.  These names come from the text in the scaler channel labels.  It was OK to acquire data with such names.  

**Looks like an assumption bug to me in the export code.**  Either of these places:

* `databroker/headersource/core.py`
* `databroker/headersource/sqlite.py`

In [8]:
list(h.events())[-1]

{'descriptor': '56aa28b1-621b-47ba-8d19-0ae05b30088e',
 'uid': '7b898a27-fe32-4a21-b852-ae2537aa04cb',
 'data': {'mca_preset_real_time': 0.25,
  'mca_elapsed_real_time': 0.25,
  'mca_spectrum': array([0, 0, 0, ..., 0, 0, 0]),
  'Time': 2500000.0,
  'IC0_D': 33.0,
  'PIN': 1.0,
  'NaI': 1.0,
  'Fwd prompt': 0.0,
  'NFS delayed': 0.0,
  'NRIXS prompt': 0.0,
  'NRIXS Delayed': 0.0,
  'scaler_time': 0.25,
  'neat_stage_y': 1.0,
  'neat_stage_y_user_setpoint': 1.0,
  'neat_stage_x': 4.0,
  'neat_stage_x_user_setpoint': 4.0},
 'timestamps': {'mca_preset_real_time': 1539893487.988774,
  'mca_elapsed_real_time': 1539893487.988774,
  'mca_spectrum': 1539893487.988774,
  'Time': 1539893487.822108,
  'IC0_D': 1539893487.822108,
  'PIN': 1539893487.822108,
  'NaI': 1539893487.822108,
  'Fwd prompt': 1539893487.822108,
  'NFS delayed': 1539893487.822108,
  'NRIXS prompt': 1539893487.822108,
  'NRIXS Delayed': 1539893487.822108,
  'scaler_time': 1539893487.822108,
  'neat_stage_y': 1539893477.638775,
  'neat_stage_y_user_setpoint': 1539893477.638775,
  'neat_stage_x': 1539893487.572108,
  'neat_stage_x_user_setpoint': 1539893487.572108},
 'time': 1539893488.0287702,
 'seq_num': 12,
 'filled': {}}